In [5]:
import json
import folium
import ee
ee.Authenticate()
ee.Initialize(project="ee-moethirizun97")

In [2]:
# Define the GLO-30 dataset
glo30 = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30")

In [7]:
# Get the first 5 images in the collection
subset = glo30.limit(5)

# Retrieve metadata for the subset
info = subset.getInfo()

# Print keys in the metadata dictionary
print(info.keys())

# Pretty print the metadata
print(json.dumps(info, indent=2))


dict_keys(['type', 'bands', 'version', 'id', 'features'])
{
  "type": "ImageCollection",
  "bands": [],
  "version": 1644374920267356,
  "id": "projects/sat-io/open-datasets/GLO-30",
  "features": [
    {
      "type": "Image",
      "bands": [
        {
          "id": "b1",
          "data_type": {
            "type": "PixelType",
            "precision": "float"
          },
          "dimensions": [
            3600,
            3600
          ],
          "crs": "EPSG:4326",
          "crs_transform": [
            0.0002777777777777778,
            0,
            5.9998611111111115,
            0,
            -0.0002777777777777778,
            1.000138888888889
          ]
        }
      ],
      "version": 1644306311292570.0,
      "id": "projects/sat-io/open-datasets/GLO-30/Copernicus_DSM_COG_10_N00_00_E006_00_DEM",
      "properties": {
        "num_bands": 1,
        "id_no": "Copernicus_DSM_COG_10_N00_00_E006_00_DEM",
        "ysize": 3600,
        "system:footprint": {
  

In [6]:
# Mosaic the DEM dataset and set visualization parameters
elev = glo30.mosaic().setDefaultProjection('EPSG:3857', None, 30)

# Define visualization parameters
elevation_palette = ['006600', '002200', 'fff700', 'ab7634', 'c4d0ff', 'ffffff']
vis_params = {
    'min': 1,
    'max': 3000,
    'palette': elevation_palette
}

# Function to add EE overlay to Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a Folium map centered on Australia
my_map = folium.Map(location=[-25, 135], zoom_start=4)

# Add the DEM layer to the map
my_map.add_ee_layer(elev, vis_params, 'GLO-30 Elevation')

# Add a layer control panel
my_map.add_child(folium.LayerControl())

# Display the map
my_map


In [9]:
# Load the GLO-30 dataset
glo30 = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30")

# Mosaic the DEM dataset
elev = glo30.mosaic()

# Load global administrative boundaries and filter for Australia
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
australia = countries.filter(ee.Filter.eq('ADM0_NAME', 'Australia'))

# Export the DEM data for Australia as a GeoTIFF
task = ee.batch.Export.image.toDrive(
    image=elev,
    description="GLO30_Australia",
    region=australia.geometry().bounds().getInfo()['coordinates'],  # Automatically get bounds
    scale=30,  # 30-meter resolution
    crs="EPSG:4326",  # Optional: Use EPSG:4326 or another CRS
    maxPixels=1e13
)

# Start the export task
task.start()

print("Export task started. Check your Google Drive for the file.")


Export task started. Check your Google Drive for the file.


In [ ]:
import os
from osgeo import gdal

# Define the directory containing the raster tiles
tiles_dir = "data-science-for-esm/data/lulc"

# Get a list of all .tif files in the directory
tile_files = [os.path.join(tiles_dir, f) for f in os.listdir(tiles_dir) if f.endswith(".tif")]

# Define the output file path for the merged raster
output_file = os.path.join(tiles_dir, "australia_lulc.tif")

# Merge all the tiles into a single file
gdal.Warp(output_file, tile_files)

print(f"Merged file saved as: {output_file}")
